In [3]:
'''
K Nearest Neighbors

>>> data_NF = np.asarray([
...     [1, 0],
...     [0, 1],
...     [-1, 0],
...     [0, -1]])
>>> query_QF = np.asarray([
...     [0.9, 0],
...     [0, -0.9]])

Example Test K=1
----------------
# Find the single nearest neighbor for each query vector
>>> neighb_QKF = calc_k_nearest_neighbors(data_NF, query_QF, K=1)
>>> neighb_QKF.shape
(2, 1, 2)

# Neighbor of [0.9, 0]
>>> neighb_QKF[0]
array([[1., 0.]])

# Neighbor of [0, -0.9]
>>> neighb_QKF[1]
array([[ 0., -1.]])

Example Test K=3
----------------
# Now find 3 nearest neighbors for the same queries
>>> neighb_QKF = calc_k_nearest_neighbors(data_NF, query_QF, K=3)
>>> neighb_QKF.shape
(2, 3, 2)

# Neighbor of [0.9, 0]
>>> neighb_QKF[0]
array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0., -1.]])

# Neighbor of [0, -0.9]
>>> neighb_QKF[1]
array([[ 0., -1.],
       [ 1.,  0.],
       [-1.,  0.]])
'''
import numpy as np

def calc_k_nearest_neighbors(data_NF, query_QF, K=1):
    ''' Compute and return k-nearest neighbors under Euclidean distance

    Args
    ----
    data_NF : 2D np.array, shape = (n_examples, n_feats) == (N, F)
        Each row is a feature vector for one example in dataset
    query_QF : 2D np.array, shape = (n_queries, n_feats) == (Q, F)
        Each row is a feature vector whose neighbors we want to find
    K : int, must satisfy K >= 1 and K <= n_examples aka N
        Number of neighbors to find per query vector

    Returns
    -------
    neighb_QKF : 3D np.array, (n_queries, n_neighbors, n_feats) == (Q, K, F)
        Entry q,k is feature vector of k-th nearest neighbor of the q-th query
        If two vectors are equally close, then we break ties by taking the one
        appearing first in row order in the original data_NF array
    '''
        
    # create the array which will hold the KNN
    neighb_QKF = np.zeros(shape=(query_QF.shape[0], K, query_QF.shape[1]))
    
    q_ctr = 0
    # for every query point
    for query in query_QF:
        # store its coordinates
        x_q = query[0]
        y_q = query[1]
        
        # create a N x 2 2D-array to hold the index in data_NF [0]
        # and the distance from the query pt. to data pt. [1]
        distances = np.zeros((data_NF.shape[0], data_NF.shape[1]))

        counter = 0
        for data in data_NF:
            x_f = data[0]
            y_f = data[1]
            
            # calculate distance from data pt. to query pt. 
            distance = ((x_q - x_f) ** 2 + (y_q - y_f) ** 2) ** 0.5
            
            # insert (index in data_NF, distance)
            distances[counter] = [counter, float(distance)] 
            counter = counter + 1
        
        print(f"Distance for query {q_ctr}: {distances}")
        # merge Sort to sort all the distances based off
        # of their distances (query pt. to data pt.)
        mergesort(distances, 0, distances.shape[0] - 1)

        # insert only K point coordinates into return array
        # in increasing order
        for i in range(0, K):
            data_key = int(distances[i, 0])
            neighb_QKF[q_ctr, i] = data_NF[data_key]
        
        # look at next query point
        q_ctr += 1
        
    return neighb_QKF

def mergesort(arr, l, r):
    if (l < r):
        # recursively call 
        m = (l + r) // 2
        mergesort(arr, l, m)
        mergesort(arr, m + 1, r)
        merge(arr, l, m, r)

# l = left index
# m = middle index
# r = right index
# arr = array of distances
def merge(arr, l, m, r):
    # create temp left / right arrays
    L = np.zeros((m - l + 1, 2))
    R = np.zeros((r - m, 2))
    
    # copy distances to L
    lidx = 0
    for i in range(l, m + 1):
        L[lidx] = arr[i]
        lidx += 1
        
    # copy distances to R
    ridx = 0
    for k in range(m + 1, r + 1):
        R[ridx] = arr[k]
        ridx += 1

    # merge L and R back into arr
    rsize = np.shape(R)[0]
    rctr = 0
    lsize = np.shape(L)[0]
    lctr = 0
    actr = l
    
    # expend the distances (in increasing order) until one array
    # is completely done
    while lctr < lsize and rctr < rsize:
        # left array has smaller value
        if L[lctr][1] <= R[rctr][1]:
            arr[actr] = L[lctr]
            lctr += 1
        # right array has smaller value
        else:
            arr[actr] = R[rctr]
            rctr += 1
        actr += 1 
        
    # left didn't expend all distances
    while lctr < lsize:
        arr[actr] = L[lctr]
        lctr += 1
        actr += 1
    
    # right didn't expend all distances
    while rctr < rsize:
        arr[actr] = R[rctr]
        rctr += 1
        actr += 1

In [4]:
data_NF = np.asarray([[1, 0], [0, 1], [-1, 0], [0, -1]])
query_QF = np.asarray([[0.9, 0], [0, -0.9]])
calc_k_nearest_neighbors(data_NF, query_QF, K=1)

Distance for query 0: [[0.        0.1      ]
 [1.        1.3453624]
 [2.        1.9      ]
 [3.        1.3453624]]
Distance for query 1: [[0.        1.3453624]
 [1.        1.9      ]
 [2.        1.3453624]
 [3.        0.1      ]]


array([[[ 1.,  0.]],

       [[ 0., -1.]]])

In [6]:
print(L[0])

[0. 0.]


In [7]:
L[0] = [0, 5]

In [8]:
print(L)

[[0. 5.]
 [0. 0.]
 [0. 0.]]


In [9]:
print(L[0])

[0. 5.]


In [10]:
np.shape(L)

(3, 2)

In [11]:
print(np.shape(L)[0])

3


In [20]:
def mergesort(arr, l, r):
    if (l < r):
        m = (l + r) // 2
#         print(f"------- l: {l},  m: {m},  r: {r} ---------")
        mergesort(arr, l, m)
        mergesort(arr, m + 1, r)
        merge(arr, l, m, r)

# l = left index
# m = middle index
# r = right index
# arr = array of distances
def merge(arr, l, m, r):
#     print(f"--------- Start of merge arr: {arr}")
    # create temp distance arrays
    L = np.zeros((m - l + 1, 2))
    R = np.zeros((r - m, 2))
    
    # copy distances from distances to L and R
#     print(f"From {l} to {m + 1}")
    lidx = 0
    for i in range(l, m + 1):
        L[lidx] = arr[i]
        lidx += 1
        
#     print(f"From {m + 1} to {r + 1}")
    ridx = 0
    for k in range(m + 1, r + 1):
        R[ridx] = arr[k]
        ridx += 1
    
        
#     print(f"Copied L: {L}")
#     print(f"Copied R: {R}")
    # merge L and R back into arr
    rsize = np.shape(R)[0]
    rctr = 0
    lsize = np.shape(L)[0]
    lctr = 0
    actr = l
    
    while lctr < lsize and rctr < rsize:
#         print("while loop")
        if L[lctr][1] <= R[rctr][1]:
            arr[actr] = L[lctr]
#             print(f"{L[lctr, 1]} added in l")
            lctr += 1
        else:
            arr[actr] = R[rctr]
#             print(f"{R[rctr, 1]} added in r")
            rctr += 1
        actr += 1 
        
    # whichever array didn't finish, fills the rest
    while lctr < lsize:
#         print("lctr < lsize")
        arr[actr] = L[lctr]
#         print(f"{L[lctr, 1]} added later")
        lctr += 1
        actr += 1
        
    while rctr < rsize:
#         print("rctr < rsize")
        arr[actr] = R[rctr]
#         print(f"{R[rctr, 1]} added later")
        rctr += 1
        actr += 1
    
#     print(f"End of merge arr: {arr}")
        

In [21]:
array = [[8, 16], [1, 15], [2, 19], [4, 13], [5, 7]]
mergesort(array, 0, 4)
print("------------------------------------")
print(type(array[1]))
print(array)

------------------------------------
<class 'numpy.ndarray'>
[array([5., 7.]), array([ 4., 13.]), array([ 1., 15.]), array([ 8., 16.]), array([ 2., 19.])]


In [17]:
for z in range(1,1):
    print("hello")

In [155]:
((1 - 0.9) ** 2 + (0 - 0) ** 2) ** 0.5

0.09999999999999998